In [1]:
#app.R
#load libraries
library(shiny)
library(xts)
library(TTR)
library(jsonlite)

Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Attaching package: 'jsonlite'

The following object is masked from 'package:shiny':

    validate



In [2]:
#Selected States
selstates <- c("Colorado","Texas","Missouri")

In [3]:
tail(Stateagg("Colorado",T,3000000))

ERROR: Error in Stateagg("Colorado", T, 3e+06): could not find function "Stateagg"


In [4]:


#function to calculate change difference
diffcalc <- function(df.xts) {
    diff = diff(df.xts)
    return(diff)
}

#function to calculate moving average
smacalc <- function(diff,l=7) {
    diff.ma = SMA(diff,l)
    return(diff.ma)
}

In [5]:
#Function to return data for individual states, aggregated
Stateagg <- function(State,Death = FALSE,pop = 0) {
    if (Death == FALSE) {
        url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
        df = read.csv(url, check.names=FALSE)
        df = df[df$Province_State == State,]
        df = df[df$Lat > 0,]
        agg = t(aggregate(. ~ Province_State, df, sum)[-c(2,3,4,5,6,7,8,9,10,11)])
    } else {
        url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
        df = read.csv(url, check.names=FALSE)
        df = df[df$Province_State == State,]
        df = df[df$Lat > 0,]
        agg = t(aggregate(. ~ Province_State, df, sum)[-c(2,3,4,5,6,7,8,9,10,11,12)])
    }
    agg.df = as.data.frame(agg[-1,])
    colnames(agg.df) = agg[1,]
    agg.df$Date = as.Date(rownames(agg.df), format = "%m/%e/%y")
    agg.df = agg.df[,c(ncol(agg.df),1:(ncol(agg.df)-1))]
    if (pop == 0) {
        agg.df[,2] = as.numeric(as.character(agg.df[,2]))
        agg.xts = xts(agg.df[,2], order.by=agg.df[,1])
        if (Death == FALSE) {
            names(agg.xts) <- "Cases"
        } else {
            names(agg.xts) <- "Deaths"
        }
    } else {
        agg.df[,2] = as.numeric(as.character(agg.df[,2]))
        agg.df[,2] <- (agg.df[,2]/pop)*100000
        agg.xts = xts(agg.df[,2], order.by=agg.df[,1])
        if (Death == FALSE) {
            names(agg.xts) <- "Cases_per_100k"
        } else {
            names(agg.xts) <- "Deaths_per_100k"
        }
    }    
    
    
    #agg.diff = diff(agg.xts)
    #agg.diff.ma = SMA(agg.diff,l)
    #results = list("df" = agg.df, "xts" = agg.xts, "diffs" = agg.diff, "sma" = agg.diff.ma)
    return(agg.xts)
}

#Function to return county level data for individual states
State <- function(State,Death = FALSE) {
    if (Death == FALSE) {
        url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
    } else {
        url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
    }
    df = read.csv(url, check.names=FALSE)
    df = df[df$Province_State == State,]
    df = df[df$Lat > 0,]
    df.t = t(df[-c(1,2,3,4,5,6,7,8,9,10,11)])
    df.df = as.data.frame(df.t[-1,])
    colnames(df.df) = df$Admin2
    df.df$Date = as.Date(rownames(df.df), format = "%m/%e/%y")
    df.df = df.df[,c(ncol(df.df),1:(ncol(df.df)-1))]
    #df.df[,c(-1)] = as.numeric(as.character(df.df[,c(-1)]))
    #df.xts = xts(df.df[,-1], order.by=df.df[,1])
    #df.diff = diff(df.xts)
    #df.diff.ma = SMA(df.diff,l)
    #results = list("df" = df.df, "xts" = df.xts, "diffs" = df.diff, "sma" = df.diff.ma)
    return(df.df)
}

#Function to return county xts
County <- function(State,County,Death = FALSE, pop = 0) {
    sdf <- State(State, Death)
    if (pop == 0) {
        cdf <- sdf[,c("Date",County)]
        cdf.xts = xts(cdf[,2], order.by=cdf[,1])
        if (Death == FALSE) {
            names(cdf.xts) <- "Cases"
        } else {
            names(cdf.xts) <- "Deaths"
        }
    } else {
        cdf <- sdf[,c("Date",County)]
        cdf[,2] <- (cdf[,2]/pop)*100000
        cdf.xts = xts(cdf[,2], order.by=cdf[,1])
        if (Death == FALSE) {
            names(cdf.xts) <- "Cases_per_100k"
        } else {
            names(cdf.xts) <- "Deaths_per_100k"
        }
    }
    
    
    return(cdf.xts)
}

In [ ]:
#Function to return a single state or county to pass to .js
Singletojs <- function(df) {
    myf <- function(x) { sprintf("[\"%s\",%s]",x[1],x[2])}
    df$Date <- as.character(df$Date)
    df <- df[seq(dim(df)[1],1),]
    cdata <- apply(df,1,myf)
    msg_str <- paste0('[',toString(cdata),']')
    return(msg_str)
}

In [ ]:
#Loop that creates data lists for each selected states
for (i in selstates) {
    state.COVID.agg <- Stateagg(i)
    assign(paste0(i,".COVID.AGG"), state.COVID.agg)
    state.COVID <- State(i)
    assign(paste0(i,".COVID"), state.COVID)
    assign(paste0(i,".Counties"), colnames(state.COVID$df[-1]))
}

In [6]:
#Funtion to return df from list item
itemtodf <- function(x) {
    df = as.data.frame(x)
    if ("Date" %in% colnames(df)) {
        return(df)
    } else {
        df <- cbind(Date = rownames(df), df)
        rownames(df) <- 1:nrow(df)
        return(df)
    }
}

In [7]:
#Loop to pull most recent summary data
for (i in 0:30) {
    USSUM <- tryCatch({
        date = as.character(format(as.Date(Sys.Date()-i), format="%m-%d-%Y"))
        url = paste0("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/",
                     date,".csv")
        read.csv(url, check.names=FALSE)
    }, warning = function(w) {w})
    if(inherits(USSUM, "warning")) next else break
}
USSUM.sel <- USSUM[USSUM$Province_State %in% selstates,]

In [9]:
toString(toJSON(USSUM))





[1] "[{\"Province_State\":\"Alabama\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":32.3182,\"Long_\":-86.9023,\"Confirmed\":43953,\"Deaths\":1007,\"Recovered\":22082,\"Active\":20864,\"FIPS\":1,\"Incident_Rate\":896.4173,\"People_Tested\":449886,\"People_Hospitalized\":2909,\"Mortality_Rate\":2.2911,\"UID\":84000001,\"ISO3\":\"USA\",\"Testing_Rate\":9175.3829,\"Hospitalization_Rate\":6.6184},{\"Province_State\":\"Alaska\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":61.3707,\"Long_\":-152.4044,\"Confirmed\":1134,\"Deaths\":16,\"Recovered\":544,\"Active\":574,\"FIPS\":2,\"Incident_Rate\":155.0144,\"People_Tested\":122732,\"Mortality_Rate\":1.4109,\"UID\":84000002,\"ISO3\":\"USA\",\"Testing_Rate\":16777.0951},{\"Province_State\":\"American Samoa\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":-14.271,\"Long_\":-170.132,\"Confirmed\":0,\"Deaths\":0,\"Active\":0,\"FIPS\":60,\"Incident_Rate\":0,\"People_Tested\":696,\"UID\":16,\"ISO3\":\"ASM\",\"Testing_Rate\":1250.8762},{\"Province_State\":\"Arizona\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":33.7298,\"Long_\":-111.4312,\"Confirmed\":98103,\"Deaths\":1825,\"Recovered\":11361,\"Active\":84917,\"FIPS\":4,\"Incident_Rate\":1347.8062,\"People_Tested\":604362,\"People_Hospitalized\":5161,\"Mortality_Rate\":1.8603,\"UID\":84000004,\"ISO3\":\"USA\",\"Testing_Rate\":8303.1391,\"Hospitalization_Rate\":5.2608},{\"Province_State\":\"Arkansas\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":34.9697,\"Long_\":-92.3731,\"Confirmed\":23814,\"Deaths\":287,\"Recovered\":16726,\"Active\":6801,\"FIPS\":5,\"Incident_Rate\":789.1169,\"People_Tested\":338893,\"People_Hospitalized\":1536,\"Mortality_Rate\":1.2052,\"UID\":84000005,\"ISO3\":\"USA\",\"Testing_Rate\":11229.7883,\"Hospitalization_Rate\":6.45},{\"Province_State\":\"California\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":36.1162,\"Long_\":-119.6816,\"Confirmed\":264681,\"Deaths\":6373,\"Active\":258308,\"FIPS\":6,\"Incident_Rate\":669.8712,\"People_Tested\":4680138,\"Mortality_Rate\":2.4078,\"UID\":84000006,\"ISO3\":\"USA\",\"Testing_Rate\":11844.7853},{\"Province_State\":\"Colorado\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":39.0598,\"Long_\":-105.3111,\"Confirmed\":34048,\"Deaths\":1701,\"Recovered\":4568,\"Active\":27779,\"FIPS\":8,\"Incident_Rate\":591.2409,\"People_Tested\":350717,\"People_Hospitalized\":5550,\"Mortality_Rate\":4.9959,\"UID\":84000008,\"ISO3\":\"USA\",\"Testing_Rate\":6090.1733,\"Hospitalization_Rate\":16.3005},{\"Province_State\":\"Connecticut\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":41.5978,\"Long_\":-72.7554,\"Confirmed\":46717,\"Deaths\":4335,\"Recovered\":8210,\"Active\":34172,\"FIPS\":9,\"Incident_Rate\":1310.3293,\"People_Tested\":499669,\"People_Hospitalized\":10411,\"Mortality_Rate\":9.2793,\"UID\":84000009,\"ISO3\":\"USA\",\"Testing_Rate\":14014.8325,\"Hospitalization_Rate\":22.2852},{\"Province_State\":\"Delaware\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":39.3185,\"Long_\":-75.5071,\"Confirmed\":12128,\"Deaths\":512,\"Recovered\":6740,\"Active\":4876,\"FIPS\":10,\"Incident_Rate\":1245.4763,\"People_Tested\":123941,\"Mortality_Rate\":4.2216,\"UID\":84000010,\"ISO3\":\"USA\",\"Testing_Rate\":12728.0327},{\"Province_State\":\"Diamond Princess\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Confirmed\":49,\"Deaths\":0,\"Active\":49,\"FIPS\":88888,\"Mortality_Rate\":0,\"UID\":84088888,\"ISO3\":\"USA\"},{\"Province_State\":\"District of Columbia\",\"Country_Region\":\"US\",\"Last_Update\":\"2020-07-06 04:34:10\",\"Lat\":38.8974,\"Long_\":-77.0268,\"Confirmed\":10482,\"Deaths\":559,\"Recovered\":1542,\"Active\":8381,\"FIPS\":11,\"Incident_Rate\":1485.2306,\"People_Tested\":105993,\"Mortality_Rate\":5.333,\"UID\":84000011,\"IS

In [ ]:
USSUM.sel[-c(2,3,4,5,10,13,15,16,18)]

In [ ]:
library(jsonlite)
jlist <- list()
USSUM.sel.sub <- USSUM.sel[-c(2,3,4,5,10,13,15,16,18)]
for (i in 1:length(colnames(USSUM.sel.sub))) {
    item = colnames(USSUM.sel.sub)[i]
    jlist[[i]] <- c(item,USSUM.sel.sub[,toString(item)])
}
jdf <- toJSON(jlist)
toString(jdf)

In [ ]:
Singletojs <- function(df) {
    myf <- function(x) { sprintf("[\"%s\",%s]",x[1],x[2])}
    df$Date <- as.character(df$Date)
    df <- df[seq(dim(df)[1],1),]
    cdata <- apply(df,1,myf)
    msg_str <- paste0('[',toString(cdata),']')
    return(msg_str)
}

Listitemtodf <- function(x) {
    df = as.data.frame(x)
    if ("Date" %in% colnames(df)) {
        return(df)
    } else {
        df <- cbind(Date = rownames(df), df)
        rownames(df) <- 1:nrow(df)
        return(df)
    }
}



In [8]:
#Singletojs(na.omit(Listitemtodf(Colorado.COVID.AGG$diffs)))
#Singletojs(na.omit(SMA(Listitemtodf(Colorado.COVID.AGG$diffs)),7))
#Singletojs(na.omit(Listitemtodf(SMA(Colorado.COVID.AGG$diffs,7))))
#graphdf <- na.omit(cbind(Colorado.COVID.AGG$diffs, SMA(Colorado.COVID.AGG$diffs,7),SMA(Colorado.COVID.AGG$diffs,14)))
#colnames(graphdf) <- c("Count","7DayMA","14DayMA")
#tail(graphdf)

graphfunc <- function(x) {
    graphdf <- na.omit(cbind(x, SMA(x,7),SMA(x,14)))
    colnames(graphdf) <- c("Count","7DayMA","14DayMA")
    return(graphdf)
}

In [ ]:
jlist <- list()
USSUM.sel.sub <- USSUM.sel[-c(2,3,4,5,10,13,15,16,18)]
USSUM.sel.sub$Province_State <- as.character(USSUM.sel.sub$Province_State)
jlist[[1]] <- c("Province_State",USSUM.sel.sub$Province_State)
for (i in 2:length(colnames(USSUM.sel.sub))) {
    item = colnames(USSUM.sel.sub)[i]
    jlist[[i]] <- c(item,as.numeric(USSUM.sel.sub[,item]))
}
jdf <- toJSON(jlist)
toString(jdf)


In [4]:
clicked_NAME <- "Colorado"
clicked_CO <- "Denver"
pop_ST <- 600000

In [10]:
st = Stateagg(clicked_NAME)
std = Stateagg(clicked_NAME, T)
stpop = Stateagg(clicked_NAME, F, pop_ST)
stdpop = Stateagg(clicked_NAME, T, pop_ST)
stcd = cbind(st, std, stpop, stdpop)
st.js = toString(toJSON(itemtodf(stcd)))

In [6]:
stcnty = County(clicked_ST,clicked_CO)
stcntyd = County(clicked_ST,clicked_CO,T)
stcntym = cbind(stcnty,stcntyd)
stcn.js = toString(toJSON(itemtodf(stcntym)))          

ERROR: Error in County(clicked_ST, clicked_CO): object 'clicked_ST' not found


In [11]:
co = Stateagg("Colorado")
mo = Stateagg("Missouri")
tx = Stateagg("Texas")
stcases <- cbind(co,mo,tx)
names(stcases) <- c("Colorado","Missouri","Texas")
stcases.js = toString(toJSON(itemtodf(stcases)))

cod = Stateagg("Colorado", T)
mod = Stateagg("Missouri", T)
txd = Stateagg("Texas", T)
stdeaths <- cbind(cod,mod,txd)
names(stdeaths) <- c("Colorado","Missouri","Texas")
stdeaths.js = toString(toJSON(itemtodf(stdeaths)))


In [12]:
stcasdiff = diffcalc(stcases)
stcasdiff.js = toString(toJSON(itemtodf(stcasdiff)))
stdeaddiff = diffcalc(stdeaths)
stdeaddiff.js = toString(toJSON(itemtodf(stdeaddiff)))

In [23]:
as.data.frame(lapply(stcasdiff,function(x){SMA(x,7)}))

,SMA,SMA.1,SMA.2
2020-01-22,NA,NA,NA
2020-01-23,NA,NA,NA
2020-01-24,NA,NA,NA
2020-01-25,NA,NA,NA
2020-01-26,NA,NA,NA
2020-01-27,NA,NA,NA
2020-01-28,NA,NA,NA
2020-01-29,0,0,0
2020-01-30,0,0,0
2020-01-31,0,0,0
